In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from nltk.corpus import stopwords
import string
import re
from bs4 import BeautifulSoup
import torch
import nltk
from sklearn.model_selection import KFold
import numpy as np

/home/emil/Desktop/diplomski/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/emil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df_fake = pd.read_csv("input/Fake.csv")
df_true = pd.read_csv("input//True.csv")

In [4]:
df_fake["label"] = 0
df_true["label"] = 1

In [5]:
df = pd.concat([df_fake, df_true], axis=0)

In [6]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [7]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

def preprocess_text(text):

    def remove_reuters_prefix(text):
        pattern = r'^[\s\S]*?\(reuters\) - '
        return re.sub(pattern, '', text)

    def strip_html(text):
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()

    def remove_square_brackets(text):
        return re.sub('\[[^]]*\]', '', text)

    def remove_urls(text):
        return re.sub(r'http\S+', '', text)

    def remove_stopwords(text):
        final_text = []
        for i in text.split():
            if i.strip().lower() not in stop:
                final_text.append(i.strip())
        return " ".join(final_text)

    text = text.lower()
    text = remove_reuters_prefix(text)
    text = strip_html(text)
    text = remove_square_brackets(text)
    text = remove_urls(text)
    text = remove_stopwords(text)

    return text


In [8]:
df['text'] = df['text'].apply(preprocess_text)

/tmp/ipykernel_63328/2048953883.py:12: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
/tmp/ipykernel_63328/2048953883.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [9]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [10]:
df = df.filter(items=['text', 'label'])

In [11]:
df.head()

,text,label
0,donald trump wish americans happy new year lea...,0
1,house intelligence committee chairman devin nu...,0
2,"friday, revealed former milwaukee sheriff davi...",0
3,"christmas day, donald trump announced would ba...",0
4,pope francis used annual christmas day message...,0


In [12]:
df = df.sample(frac = 1)
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [13]:
df.head()

,text,label
0,moscow-based antivirus software maker kaspersk...,1
1,say leftists media completely unhinged would u...,0
2,u.s. president donald trump arrived israel mon...,1
3,tennessee woman obviously bought left playbook...,0
4,"u.s. lawmakers, alarmed foreign entities used ...",1


In [14]:
texts = df['text'].tolist()
labels = df['label'].tolist()

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
def tokenize_function(data):
    return tokenizer(data['text'], padding="max_length", truncation=True)

In [17]:
dataset = Dataset.from_pandas(df)

In [18]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 44898/44898 [02:56<00:00, 253.98 examples/s]


In [19]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [20]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [21]:
results = []

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [23]:
for fold, (train_index, val_index) in enumerate(kf.split(tokenized_dataset)):

    print(f'Fold {fold+1}')
    train_dataset = tokenized_dataset.select(train_index)
    val_dataset = tokenized_dataset.select(val_index)
    
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
    
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold+1}',
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f'./logs_fold_{fold+1}',
        logging_steps=50,
        save_total_limit=3,
        evaluation_strategy="steps",
        eval_steps=2000,
        save_steps=2000,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )
    
    trainer.train()
    eval_result = trainer.evaluate()
    results.append(eval_result)

    model.save_pretrained(f'./model_fold_{fold+1}')
    tokenizer.save_pretrained(f'./model_fold_{fold+1}')

 60%|█████▉    | 8050/13470 [2:17:17<1:19:42,  1.13it/s]

{'loss': 0.0001, 'grad_norm': 0.002089222427457571, 'learning_rate': 2.089437162683115e-05, 'epoch': 1.79}


 60%|██████    | 8100/13470 [2:18:02<1:19:18,  1.13it/s]

{'loss': 0.0001, 'grad_norm': 0.0017749660182744265, 'learning_rate': 2.0701619121048574e-05, 'epoch': 1.8}


 61%|██████    | 8150/13470 [2:18:46<1:16:57,  1.15it/s]

...

100%|█████████▉| 13450/13470 [3:45:56<00:17,  1.15it/s]

{'loss': 0.0001, 'grad_norm': 0.002184685319662094, 'learning_rate': 7.710100231303007e-08, 'epoch': 3.0}


100%|██████████| 13470/13470 [3:46:14<00:00,  1.01s/it]


{'train_runtime': 13574.1496, 'train_samples_per_second': 7.938, 'train_steps_per_second': 0.992, 'train_loss': 0.03226191357037249, 'epoch': 3.0}


100%|██████████| 1123/1123 [04:36<00:00,  4.06it/s]


In [24]:
average_accuracy = np.mean([result['eval_accuracy'] for result in results])
print(f'Average Accuracy: {average_accuracy}')

Average Accuracy: 0.9985077142715454
